In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import numpy as np

def get_batch(T,batch_size):
    values = torch.rand(T, batch_size, requires_grad=False)
    indices = torch.zeros_like(values)
    half = int(T / 2)
    for i in range(batch_size):
        half_1 = np.random.randint(half)
        hals_2 = np.random.randint(half, T)
        indices[half_1, i] = 1
        indices[hals_2, i] = 1

    data = torch.stack((values, indices), dim=-1)
    targets = torch.mul(values, indices).sum(dim=0)
    return data, targets

In [3]:
from torch import nn
import torch
from torch.autograd import Variable
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class coRNNCell(nn.Module):
    def __init__(self, n_inp, n_hid, dt, gamma, epsilon):
        super(coRNNCell, self).__init__()
        self.dt = dt
        self.gamma = gamma
        self.epsilon = epsilon
        self.i2h = nn.Linear(n_inp + n_hid + n_hid, n_hid)
        self.bn = nn.BatchNorm1d(n_hid) 

    def forward(self,x,hy,hz):
        hz = hz + self.dt * (torch.tanh(self.bn(self.i2h(torch.cat((x, hz, hy),1))))
                                   - self.gamma * hy - self.epsilon * hz)
        hy = hy + self.dt * hz

        return hy, hz

class coRNN(nn.Module):
    def __init__(self, n_inp, n_hid, n_out, dt, gamma, epsilon):
        super(coRNN, self).__init__()
        self.n_hid = n_hid
        self.cell = coRNNCell(n_inp,n_hid,dt,gamma,epsilon)
        self.readout = nn.Linear(n_hid, n_out)
        self.bn = nn.BatchNorm1d(n_hid) 

    def forward(self, x):
        ## initialize hidden states
        hy = Variable(torch.zeros(x.size(1),self.n_hid)).to(device)
        hz = Variable(torch.zeros(x.size(1),self.n_hid)).to(device)

        for t in range(x.size(0)):
            hy, hz = self.cell(x[t],hy,hz)
        output = self.readout(self.bn(hy))

        return output

In [4]:
from torch import nn, optim
import torch
# import model
import torch.nn.utils
# import utils
# import argparse
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# parser = argparse.ArgumentParser(description='training parameters')

# parser.add_argument('--n_hid', type=int, default=128,
#                     help='hidden size of recurrent net')
# parser.add_argument('--T', type=int, default=100,
#                     help='length of sequences')
# parser.add_argument('--max_steps', type=int, default=60000,
#                     help='max learning steps')
# parser.add_argument('--log_interval', type=int, default=100,
#                     help='log interval')
# parser.add_argument('--batch', type=int, default=50,
#                     help='batch size')
# parser.add_argument('--batch_test', type=int, default=1000,
#                     help='size of test set')
# parser.add_argument('--lr', type=float, default=2e-2,
#                     help='learning rate')
# parser.add_argument('--dt',type=float, default=6e-2,
#                     help='step size <dt> of the coRNN')
# parser.add_argument('--gamma',type=float, default=66,
#                     help='y controle parameter <gamma> of the coRNN')
# parser.add_argument('--epsilon',type=float, default = 15,
#                     help='z controle parameter <epsilon> of the coRNN')

# args = parser.parse_args()
args = {"n_hid":128,
      "T" : 100,
      "max_steps":30000,
      "log_interval":100,
      "batch":50,
      "batch_test":1000,
      "lr":2e-2,
      "dt":6e-2,
      "gamma":66,
      "epsilon":15}

n_inp = 2
n_out = 1

model = coRNN(n_inp, args["n_hid"], n_out, args["dt"], args["gamma"], args["epsilon"]).to(device)


objective = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=args["lr"])
test_mse_500 = []
test_mse_2000 = []
test_mse_5000 = []

def test():
    model.eval()
    with torch.no_grad():
        data, label = get_batch(args["T"], args["batch_test"])
        label = label.unsqueeze(1)
        out = model(data.to(device))
        loss = objective(out, label.to(device))

    return loss.item()

def train(x,l):
    
    
    for i in range(args["max_steps"]):
#         model.train()
        data, label = get_batch(x,args["batch"])
#         print(data.shape)
#         print(label.shape)
        label = label.unsqueeze(1)
#         print(label.shape)

        optimizer.zero_grad()
        out = model(data.to(device))
        loss = objective(out, label.to(device))
        loss.backward()
        optimizer.step()
        if(i%100==0 and i!=0):
            mse_error = test()
            print('Test MSE: {:.6f}'.format(mse_error))
            l.append(mse_error)
            model.train()
            


In [5]:
train(100,test_mse_500)

Test MSE: 0.155186
Test MSE: 0.176128
Test MSE: 0.169130
Test MSE: 0.193527
Test MSE: 0.165970
Test MSE: 0.184066
Test MSE: 0.186148
Test MSE: 0.158288
Test MSE: 0.172829
Test MSE: 0.180175
Test MSE: 0.171954
Test MSE: 0.169865
Test MSE: 0.168531
Test MSE: 0.182099
Test MSE: 0.173234
Test MSE: 0.170290
Test MSE: 0.167101
Test MSE: 0.173063
Test MSE: 0.163827
Test MSE: 0.163511
Test MSE: 0.170801
Test MSE: 0.168119
Test MSE: 0.167162
Test MSE: 0.172298
Test MSE: 0.168527
Test MSE: 0.159989
Test MSE: 0.163508
Test MSE: 0.167529
Test MSE: 0.167778
Test MSE: 0.157105
Test MSE: 0.167005
Test MSE: 0.161474
Test MSE: 0.152374
Test MSE: 0.149399
Test MSE: 0.167803
Test MSE: 0.159183
Test MSE: 0.150765
Test MSE: 0.154293
Test MSE: 0.158895
Test MSE: 0.150819
Test MSE: 0.154529
Test MSE: 0.158936
Test MSE: 0.156776
Test MSE: 0.157708
Test MSE: 0.165969
Test MSE: 0.167639
Test MSE: 0.151868
Test MSE: 0.156999
Test MSE: 0.165298
Test MSE: 0.170136
Test MSE: 0.156777
Test MSE: 0.159766
Test MSE: 0.